In [5]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    przekazane=RunnablePassthrough(),
    z_dodatkiem=RunnablePassthrough.assign(pomnozone=lambda x: x["num"] * 3),
    zmodyk=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'przekazane': {'num': 1},
 'z_dodatkiem': {'num': 1, 'pomnozone': 3},
 'zmodyk': 2}

## Retrieval Example

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'